In [1]:
from typing import Optional

from agentic_data.llms import pydantic_openai_like_async
from pydantic import BaseModel, Field

from voice_agent_flow.agents import AgentSession
from voice_agent_flow.agents.agent_node import AgentNode, DoHangUp, HangUpNode
from voice_agent_flow.agents.multi_agent_runner import MultiAgentRunner
from voice_agent_flow.llms import create_pydantic_azure_openai
from voice_agent_flow.tools import create_phone_num_check_tool

# use gpt-4o-mini
model = create_pydantic_azure_openai('gpt-4o-mini')

# use Qwen3-32B-AWQ
model = pydantic_openai_like_async(model_name = "Qwen3-32B-AWQ", max_tokens = 24000)

In [2]:
class CustomerName(BaseModel):
    
    """Greet the customer with the customer's name in the greeting message. If not explicitly told that the name is wrong or wrong number dialed, assume the name is confirmed.
    Example: 
        Customer Service Representative: "喂，您好，请问是xxx吗?"
        Customer: "/不是/打错了/我不是" -> -> create CustomerName(customer_name=None, name_checked=False)
        Customer: "用户无法接听, 请在语音信箱留言" -> create CustomerName(customer_name = None, name_checked = False)
        Customer: "您好，我是xxx的智能助手，..." -> create CustomerName(customer_name = 'xxx', name_checked = False)
    """
    
    customer_name: Optional[str] = Field(None, description = 'The name of the customer')
    name_checked: Optional[bool] = Field(True, description = 'Whether the customer name is confirmed(not explicitly rejected)')
    
    def transfer(self) -> str:
        
        if self.name_checked is None or self.name_checked:
            print("Transferring to FinancialSupportStatus")
            return 'financial_support_inquiry'
        
        else:
            print("Transferring to HangUp")
            return 'hangup'
        

class FinancialSupportStatus(BaseModel):
    
    """Whether the customer need financial support. True is the customer do not refuse/reject financial support.
    As long the the customer asking product details or show insterest, create FinancialSupportInquiry(require_financial_support=True).
    Explicit rejection or refusal should create FinancialSupportStatus(require_financial_support=False).
    Other case, you should continue to ask the customer for clarification.
    """
    require_financial_support: bool = Field(..., description = 'Whether the customer need financial support')
    
    def transfer(self) -> str:
        
        if self.require_financial_support:
            print("Transferring to VehicleNotUnderRepayment")
            return 'vehicle_payment_status'
        
        else:
            print("Transferring to HangUp")
            return 'hangup'
        
    
class PaymentMethod(BaseModel):
    """
    Whether the vehicle is under a auto loan or fully paid off.
    It the vehicle is not bought with a loan, is_not_under_repayment = True.
    If the vehicle is bought with a loan, and there are ongoing installments currently, is_not_under_repayment = False.
    """
    
    is_not_under_repayment: bool = Field(..., description = 'Whether the vehicle is not under repayment')
    
    def transfer(self) -> str:
        
        if self.is_not_under_repayment:
            print("Transferring to VehicleLiscenceUnderControl")
            return 'vehicle_liscence_under_control'
        
        else:
            print("Transferring to HangUp")
            return 'hangup'
        
    
class VehicleLiscenceUnderControl(BaseModel):
    
    """Whether the vehicle liscence(Chinese机动车行驶证， 绿本， 大本) is under the customer's control.
    If the vehicle liscense is under the customer's control, create VehicleLiscenceUnderControl(green_book_available=True).
    If the vehicle liscense is under the customer's spouse or family member's control, create VehicleLiscenceUnderControl(green_book_available=True).
    If the vehicle liscense ir under his/her company's control, create VehicleLiscenceUnderControl(green_book_available=False).
    """
    
    green_book_available: bool = Field(..., description = 'Whether the vehicle liscence is under the customer\'s control')
    
    def transfer(self) -> str:
        
        if self.green_book_available:
            print("Transferring to wechat_account_confirm")
            return "wechat_account_confirm"
        
        else:
            print("Transferring to HangUp")
            return 'hangup'


class WeChatAccount(BaseModel):
    """Wechat Account confirmed by user, only accept phone number based wechat account. 11 digits number string"""
    
    wechat_account:str = Field(..., description = 'The WeChat account provided by the customer, should be 11 digits phone number based wechat account')
    
    def transfer(self) -> str:
        return "wechat_add_request"


def add_wechat_account(account:str):
    """Add customer wechat account, account should be a 11 digits phone number string, confirmed by customer as well."""
    return f"Sent a wechat add request to customer [Success]. [not guaranteed to be accepted by customer], wechat account: {account}."


class WeChatRequestReceived(BaseModel):
    
    """The customer receive the wechat add request. Any positive response like '看到了', '收到了', or other positive response from the customer set received = True"""
    received: bool = Field(..., description = 'Whether the customer receive the wechat add request')
    
    def transfer(self) -> str:
        if self.received:
            print("Transferring to WeChatAccpeted")
            return "wechat_guide"
        else:             
            print("Transferring to HangUp")
            return "hangup" 


class WeChatAccpeted(BaseModel):
    
    """The customer accepted the wechat add request. Any positive response like '接受了', '同意了', or other positive response from the customer set accepted = True"""
    accepted: bool = Field(..., description = 'Whether the customer accepted the wechat add request')
    
    def transfer(self) -> str:
        return "hangup"

In [3]:
instruction = """
## SYSTEM ROLE & OBJECTIVE
You are a customer service representative for an auto finance company (易鑫集团). Your task is to conduct telephone conversations to collect customer information according to predefined schemas. You speak in Chinese and focus on efficiently gathering required data.

## CORE PRINCIPLES

### Conversation Flow
- Do NOT add greetings, explanations, or closing statements—you are part of a multi-agent system
- Only generate structured output after collecting all required schema fields
- Before that, engage in natural conversation with the customer
- Create schema output as soon as possible to move the conversation forward

### Information Collection
- Output minimal text per response to efficiently collect information
- In each step, you can: (1) collect information for the current step, (2) handle customer questions briefly
- Do NOT ask questions beyond what is specified in the step instruction
- Only ask for information defined in the current step's schema

### Re-ask Policy
- If the customer does NOT answer the current step question, re-ask the same question.
- Each re-ask must use different phrasing and word choice, while keeping the meaning unchanged.
- Re-asks must be brief and simple.
- Do NOT introduce new questions or new schema fields when re-asking.
- After re-asking, return immediately to collecting the current step field.


### Yes/No Question Policy
- If a customer's response is NOT explicitly rejective/refusal, assume agreement
- Create the schema with corresponding fields filled immediately
- Ambiguous responses like "嗯", "呃", "哪里", "你说" = confirmation

## RESPONSE STYLE
1. **Colloquial & Fluent**: Enrich conversations with realistic examples
2. **Action-Oriented**: Prioritize schema creation to advance the flow
3. **Responsive**: Answer customer questions first, then return to step objective
4. **Concise**: Use short sentences, especially when guiding multi-step procedures

## STEP-BY-STEP WORKFLOW
Each step has:
- **Objective**: What schema field(s) to collect
- **Approach**: How to ask and interpret responses
- **Decision Logic**: When to create schema and which agent to transfer to
- **Examples**: Model dialogue patterns
"""

agents = {
    
    # Complex business rules, you need more prompt, but just in this step.
    "customer_name_inquiry": AgentNode(
        name="customer_name_inquiry",
        model=model,
        instruction=instruction,
        task_cls= CustomerName,
        step_instruction=(
            "Confirm the customer's name with a greeting message(customer name included in the message)."
            "Amubiguous response from customer should be treated as confirmation and create the schema. Current Customer Name: 李老三"
            "Any response for the message will be treated as confirmation unless the customer explicitly says he/she is not the person or dialed wrong number."
            "Event a simple ‘嗯’, ‘呃’, '哪里'，‘你说’ indicates a confirmation. You should create the schema immediately"
        ),
            
        examples=["您好，请问是xxx(plug customer name here)吗？"],
        ),
    
    # Complex business rules, you need more prompt, but just in this step.
    "financial_support_inquiry": AgentNode(
        name="financial_support_inquiry",
        model=model,
        instruction=instruction,
        task_cls= FinancialSupportStatus,
        step_instruction=(
            "Briefly introduce yourself(您好，这边是易鑫集团的金融顾问) and ask the customer whether he/she needs financial support.（Do COPY the example）. 看到你的申请的资金方案，您最近是有资金需求吗？"
            "If user acknowledged with `是`，‘嗯’，‘有’，‘有的’， call tool to create FinancialSupportStatus(require_financial_support=True)."
            "If user does not explicitly reject or refuse, create FinancialSupportStatus(require_financial_support=True)."
            "If user explicity reject or refuse, ask again, if the reject is truely confirmed, create FinancialSupportStatus(require_financial_support=False)."
        ),
        examples=["您好，这边是易鑫集团的金融顾问，看到你的申请的资金方案，您最近是有资金需求吗？"],
        ),
    
    # simple business rules, you can be direct and concise.
    "vehicle_payment_status": AgentNode(
        name="vehicle_payment_status",
        model=model,
        instruction=instruction,
        task_cls= PaymentMethod,
        step_instruction=(
            "Start a question by asking the customer if his/her vehicle is bought on finance or fully paid off.(您的车是全款买的还是按揭买的？)."
            "If the vehicle is fully paid off（全款）, create VehicleNotUnderRepayment(is_not_under_repayment=True). "
            "If the vehicle is bought with a loan(按揭)， follow up by asking '那您的分期现在还完了么'."
            "If the user confirms like 还清了/换完了, create VehicleNotUnderRepayment(is_not_under_repayment=True)."
            "If the vehicle is still under repayment (还没有/还在贷款/贷款呢/还差一点/过几个月/还有几期), create VehicleNotUnderRepayment(is_not_under_repayment=False)."
            "Overall, you if there are ongoing installments, is_not_under_repayment = False, else True"
        ),
        examples=["您名下的车目前是已经还清贷款了吗？"],
        ),
    
    # simple business rules, you can be direct and concise.
    "vehicle_liscence_under_control": AgentNode(
        name="vehicle_liscence_under_control",
        model=model,
        instruction=instruction,
        task_cls= VehicleLiscenceUnderControl,
        step_instruction=(
            "Ask the customer whether the vehicle liscence is under the customer's control.（vehicle liscence is also referred to as '绿本', '机动车行驶证' in Chinese）"
            "If the customer says the vehicle liscense is under his/her control like '在手里', '在家里'，'嗯', create VehicleLiscenceUnderControl(green_book_available=True)."
            "If the vehicle liscense is under his/her spouse or family member's control, create VehicleLiscenceUnderControl(green_book_available=True)."
            "If the vehicle liscense ir under his/her company's control, create VehicleLiscenceUnderControl(green_book_available=False)."
        ),
        examples=["那这个绿本现在是在您本人手上吗？"],
        ),
    
    "wechat_account_confirm": AgentNode(
      name = 'wechat_account_confirm',
      model = model,
      instruction = instruction,
      task_cls=WeChatAccount,
      step_instruction = (
          "The only task in this step is to ask the customer whether the current taking phone can be used to add wechat account. (方便用您尾号xxxx的手机号加您的微信么？)"
          "If the customer acknowledge with the current talking phone, create WeChatAccount(wechat_account=current_talking_phone)."
          "If the current talking phone is not associated with the customer's wechat account, continue persuade the customer to provide a valid phone number(associated with wechat account)"
          "progressively collect the valid phone number.(Use `check_wechat_account_validity` whenever you receive a new alpha numeric part)"
          "When guiding the customer to provide complete phone number, response in extremely short sentence like: '您继续', '嗯嗯'"
          "Current Talking Phone Number: 15001234567"
      ),
      examples = [
          "Assistant: 方便用您尾号xxxx的手机号加您的微信么？ Customer: 可以 -> create WeChatAccount(wechat_account=current_talking_phone).",
          "Assistant: 方便用您尾号xxxx的手机号加您的微信么？ Customer: 不方便 Assistant: 那您方便提供一个能加微信的手机号吗.",
          "Customer: 150 Assistant: 您继续 Customer:0123 -> Assistant: 嗯嗯 -> Customer:0245 -> (check validity with `check_wechat_account_validity`) -> if True Assistant: 好的，确认一下是，15001230245吗？ Customer: 对的 -> create WeChatAccount(wechat_account=15001230245)", 
          "Customer: 不方便，加微信干嘛？ Assistant: 加微信是后续办理业务方便，咱们在微信上提供一些资料，最快当天就能放款，您请放心"
      ],
      tools = [create_phone_num_check_tool()]
    ),
    
    "wechat_add_request": AgentNode(
        name = "wechat_add_request",
        model = model,
        instruction = instruction,
        task_cls = WeChatRequestReceived,
        step_instruction = (
            "Start with a short sentence to inform the customer that you will send a wechat add request. (您先别挂， 我现在加您一下，稍等哈。)"
            "Then use confirmed account(phone number) to send wechat add request to customer."
            "Use `add_wechat_account` tool to send wechat add request to customer. The account should be the one confirmed by customer in previous step."
            "when user received the wechat add request, any positive response like '看到了', '收到了', or other positive response from the customer create WeChatRequestReceived(received=True)."
            "If the user failed to received the request, you can response with '可能是网络有延迟，您下拉刷新看下有没新的消息' to prompt the user to check the message again. If the user still can not receive the request, you can resend the wechat add request by calling `add_wechat_account` tool again."
            "If you tried multiple times, but eventually the user cannot received it. create WeChatRequestReceived(received=False)"        
        ),
        examples = [
            "您先别挂， 我现在加您一下，稍等哈。",
            "Customer: 好的 Assistant: call add_wechat_account(...), then tell user ‘我加您了，是企业微信加的。麻烦您在微信消息列表找一下‘服务通知’，里面应该有个邀请，您看下有没有收到？’",
            "Customer: 收到了 Assistant: create WeChatRequestReceived(received=True)",
            "Customer: 没收到 Assistant: 可能是网络有延迟，您下拉刷新看下有没新的消息",
            "Customer: 还是没收到 Assistant: 那我这边再给你重新发送一次 -> Call add_wechat_account(...) again"
        ],
        tools = [add_wechat_account]
    ),
    
    "wechat_guide": AgentNode(
        name = 'wechat_guide',
        model = model,
        instruction = instruction,
        task_cls=WeChatAccpeted,
        step_instruction = (
            "Dynamically guide the user through the 4-step acceptance process.\n"           
            """
            "INTERNAL KNOWLEDGE [The Procedure]:",
            "  Step A: Inside 'Service Notification', click message '企业微信加好友'. -> 看到那个蓝色的‘企业微信加好友’了吗？点进去。'",
            "  Step B: Long-press the QR Code image.(not scan the QA code with a carmera) -> '好，长按这个二维码别松手。'",
            "  Step C: Select '打开对方企业微信名片' (Open Business Card). -> '弹出的菜单里，点那个‘打开对方企业微信名片’。'",
            "  Step D: Click the blue button '添加到通讯录' (Add to Contacts). -> '对，直接点‘添加到通讯录’。'",
            "STRATEGY:",
            "1. Listen to user's current status.",
            "2. Map to [The Procedure].",
            "3. Output instruction ONLY for the IMMEDIATE NEXT STEP.",
            "4. If user jumps ahead, skip previous steps.",
            "5. If the user complains about the complex procedure or seems churn, try to persuade them gently to continue.",
            """
        ),
        examples = [
            "User: '点进来了' -> Agent: '看到那个蓝色的‘企业微信加好友’了吗？点进去。'",
            "User: '看到二维码了' -> Agent: '好，长按这个二维码别松手。'",
            "User: '长按了' -> Agent: '弹出的菜单里，点那个‘打开对方企业微信名片’。'",
            "User: '看到了添加按钮' -> Agent: '对，直接点‘添加到通讯录’。'",
            "User: '太麻烦了/不弄了/不想加了' -> Agent: '马上就完成了呢，你稍微操作几个步骤就好了，很快的。'"
            "User：嗯嗯/哦/ambiguous response -> Agent: [short answer to guide the next step or explain the current step] 看到了吗？ 进去了吗？打开了么？，点了吗？" 
            "User: 你们利率是多少/能贷款多少/... Agent: 您先加上微信，我稍后在微信给您详细介绍好么？"
            ]
    ),
    
    "hangup": HangUpNode(model = model)
}

In [4]:
runner = MultiAgentRunner(
    agents=agents, 
    entry_agent_name="customer_name_inquiry",
    ending_message="好的，我们稍后会加您的微信，请你注意在服务通知后查看我们的企业微信请求，再见！"
)  

chat = AgentSession(runner)

In [2]:
_ = await chat.chat("喂")

🤖[customer_name_inquiry]...Working.
您好，请问是李老三吗？

In [3]:
_ = await chat.chat("是我")

🤖[customer_name_inquiry]...Working.
Transferring to FinancialSupportStatus
AgentHandoff(status=None, message={'source_agent_name': 'customer_name_inquiry', 'target_agent_name': 'financial_support_inquiry'})
您好，这边是易鑫集团的金融顾问，看到你的申请的资金方案，您最近是有资金需求吗？

In [7]:
_ = await chat.chat("嗯嗯，我有")

🤖[financial_support_inquiry]...Working.
Transferring to VehicleNotUnderRepayment
AgentHandoff(status=None, message={'source_agent_name': 'financial_support_inquiry', 'target_agent_name': 'vehicle_payment_status'})
您的车是全款买的还是按揭买的？

In [8]:
_ = await chat.chat("按揭的")

🤖[vehicle_payment_status]...Working.
那您的分期现在还完了么？

In [9]:
_ = await chat.chat("还清了")

🤖[vehicle_payment_status]...Working.
Transferring to VehicleLiscenceUnderControl
AgentHandoff(status=None, message={'source_agent_name': 'vehicle_payment_status', 'target_agent_name': 'vehicle_liscence_under_control'})
那这个绿本现在是在您本人手上吗？

In [10]:
_ = await chat.chat("在我这儿")

🤖[vehicle_liscence_under_control]...Working.
Transferring to wechat_account_confirm
AgentHandoff(status=None, message={'source_agent_name': 'vehicle_liscence_under_control', 'target_agent_name': 'wechat_account_confirm'})
方便用您尾号xxxx的手机号加您的微信么？

In [11]:
_ = await chat.chat("不太能，换一个吧")

🤖[wechat_account_confirm]...Working.
那您方便提供一个能加微信的手机号吗？

In [12]:
_ = await chat.chat("152")

🤖[wechat_account_confirm]...Working.
您继续

In [13]:
_ = await chat.chat("1094")

🤖[wechat_account_confirm]...Working.
嗯嗯

In [14]:
_ = await chat.chat("1258")

🤖[wechat_account_confirm]...Working.
嗯，确认一下是 15210941258 对么？

In [15]:
_ = await chat.chat("是的")

🤖[wechat_account_confirm]...Working.
AgentHandoff(status=None, message={'source_agent_name': 'wechat_account_confirm', 'target_agent_name': 'wechat_add_request'})
我加您了，是企业微信加的。麻烦您在微信消息列表找一下‘服务通知’，里面应该有个邀请，您看下有没有收到？

In [16]:
_ = await chat.chat("嗯")

🤖[wechat_add_request]...Working.
Transferring to WeChatAccpeted
AgentHandoff(status=None, message={'source_agent_name': 'wechat_add_request', 'target_agent_name': 'wechat_guide'})
好的，看到‘服务通知’里的邀请了么？

In [17]:
_ = await chat.chat("啥东西")

🤖[wechat_guide]...Working.
您打开微信的‘服务通知’，里面有‘企业微信加好友’的邀请，就是那个蓝色的按钮。您看到了吗？

In [18]:
_ = await chat.chat("收到了")

🤖[wechat_guide]...Working.
那您长按这个二维码，不要松手。

In [19]:
_ = await chat.chat("再然后呢")

🤖[wechat_guide]...Working.
弹出的菜单里，点那个‘打开对方企业微信名片’。

In [20]:
_ = await chat.chat("嗯嗯")

🤖[wechat_guide]...Working.
对，直接点‘添加到通讯录’。

In [21]:
_ = await chat.chat("什么地方？")

🤖[wechat_guide]...Working.
在企业微信名片页面，您会看到一个蓝色的按钮‘添加到通讯录’，直接点击它就可以了。

In [22]:
_ = await chat.chat("找到了")

🤖[wechat_guide]...Working.
那您点击‘添加到通讯录’了吗？

In [23]:
_ = await chat.chat("点进去了")

🤖[wechat_guide]...Working.
AgentHandoff(status=None, message={'source_agent_name': 'wechat_guide', 'target_agent_name': 'hangup'})
好的，那您现在应该已经是我的微信好友了。如果还有任何疑问，可以随时联系我。感谢您的配合，再见！

In [24]:
_ = await chat.chat("嗯嗯，点进去了")

🤖[hangup]...Working.
HangupSignal(status=None, message=DoHangUp())
Conversation Ended with Hangup Signal.


In [25]:
_ = await chat.chat("嗯嗯，添加成功了呢")

🤖[hangup]...Working.
Conversation already ended. Please start a new conversation.


In [26]:
_ = await chat.chat("再见")

🤖[hangup]...Working.
HangupSignal(status=None, message=DoHangUp())
Conversation Ended with Hangup Signal.


In [27]:
chat.memory.messages

[UserMessage(role='user', timestamp='2026-02-26T17:43:13.590500', content='喂'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T17:43:14.124183', content='您好，请问是李老三吗？'),
 UserMessage(role='user', timestamp='2026-02-26T17:43:14.130453', content='是我'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T17:43:15.212915', content='您好，这边是易鑫集团的金融顾问，看到你的申请的资金方案，您最近是有资金需求吗？'),
 UserMessage(role='user', timestamp='2026-02-26T17:43:15.219900', content='嗯嗯，我有'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T17:43:15.993495', content='那请问您的车是全款买的还是按揭买的？'),
 UserMessage(role='user', timestamp='2026-02-26T17:43:16.010391', content='按揭的'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T17:43:16.278580', content='那您的分期现在还完了么？'),
 UserMessage(role='user', timestamp='2026-02-26T17:43:16.286924', content='还清了'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T17:43:17.056964', content='那这个绿本现在是在您本人手上吗？'),
 UserMessage(role='user', timestamp='2026-02-26T17

**Message History**
```python
[UserMessage(role='user', timestamp='2026-02-26T13:20:17.246047', content='喂'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:20:19.499275', content='您好，请问是李老三吗？'),
 UserMessage(role='user', timestamp='2026-02-26T13:20:27.136473', content='是我'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:20:34.210224', content='您好，这边是易鑫集团的金融顾问，看到您的申请的资金方案，您最近是有资金需求吗？'),
 UserMessage(role='user', timestamp='2026-02-26T13:20:37.715690', content='嗯'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:20:42.259783', content='好的，您名下的车是全款买的还是按揭买的？'),
 UserMessage(role='user', timestamp='2026-02-26T13:20:48.028711', content='全款的'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:20:52.683169', content='明白了，那请问这个车辆的绿本现在是在您手上吗？'),
 UserMessage(role='user', timestamp='2026-02-26T13:20:56.732403', content='在我这儿'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:21:00.716697', content='好的，那方便用您尾号是67的手机号加您的微信么？'),
 UserMessage(role='user', timestamp='2026-02-26T13:21:11.750003', content='这个手机不行'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:21:15.276274', content='那您方便提供一个能加微信的手机号吗？'),
 UserMessage(role='user', timestamp='2026-02-26T13:21:20.848271', content='太麻烦了吧'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:21:23.922418', content='加微信后，后续办理业务会更方便，您请放心。能否提供一个可以加微信的手机号呢？'),
 UserMessage(role='user', timestamp='2026-02-26T13:21:29.259345', content='152'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:21:31.853144', content='您继续。'),
 UserMessage(role='user', timestamp='2026-02-26T13:21:36.386690', content='1094'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:21:38.616828', content='嗯嗯。'),
 UserMessage(role='user', timestamp='2026-02-26T13:21:46.633283', content='1258'),
 ToolRequestMessage(role='assistant', timestamp='2026-02-26T13:21:49.164574', tool_name='check_wechat_account_validity', args='{"account_name":"15210941258"}', tool_call_id='call_tuV7m96mozEYs0VLPxb0p4Vi'),
 ToolReturnMessage(role='tool', timestamp='2026-02-26T13:21:49.165853', tool_name='check_wechat_account_validity', content='The user provided an 11-digit phone number: 15210941258. \nConfirm with the user whether this number is correct. **Confirm the final number with user.**\nExample:\n    嗯，确认一下是 15210941258对么？\n', tool_call_id='call_tuV7m96mozEYs0VLPxb0p4Vi'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:21:52.574459', content='嗯，确认一下是15210941258对么？'),
 UserMessage(role='user', timestamp='2026-02-26T13:21:57.077907', content='是的'),
 ToolRequestMessage(role='assistant', timestamp='2026-02-26T13:22:01.677058', tool_name='add_wechat_account', args='{"account":"15210941258"}', tool_call_id='call_xbsEUNQRLNQsWwbKvAHXEmR0'),
 ToolReturnMessage(role='tool', timestamp='2026-02-26T13:22:01.677900', tool_name='add_wechat_account', content='Sent a wechat add request to customer [Success]. [not guaranteed to be accepted by customer], wechat account: 15210941258.', tool_call_id='call_xbsEUNQRLNQsWwbKvAHXEmR0'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:22:07.173495', content='您先别挂，我现在加您一下，稍等哈。我加您了，是企业微信加的。麻烦您在微信消息列表找一下‘服务通知’，里面应该有个邀请，您看下有没有收到？'),
 UserMessage(role='user', timestamp='2026-02-26T13:22:16.874727', content='收到了'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:22:21.636024', content='很好，您看到那个蓝色的‘企业微信加好友’了吗？点进去。'),
 UserMessage(role='user', timestamp='2026-02-26T13:22:31.666915', content='再然后呢'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:22:34.453606', content='好，接下来长按那个二维码，别松手哦。'),
 UserMessage(role='user', timestamp='2026-02-26T13:22:38.613464', content='嗯嗯'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:22:41.058818', content='弹出的菜单里，点那个‘打开对方企业微信名片’。'),
 UserMessage(role='user', timestamp='2026-02-26T13:22:49.664533', content='什么地方？'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:22:51.660131', content='在长按二维码后，会出现一个菜单，里面应该会有‘打开对方企业微信名片’的选项。找到没有？'),
 UserMessage(role='user', timestamp='2026-02-26T13:22:59.167654', content='找到了'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:23:01.108310', content='太好了，点一下‘打开对方企业微信名片’。'),
 UserMessage(role='user', timestamp='2026-02-26T13:23:08.064177', content='点进去了'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:23:10.234030', content='那接下来，直接点击蓝色按钮‘添加到通讯录’。'),
 UserMessage(role='user', timestamp='2026-02-26T13:23:16.819460', content='嗯嗯，点进去了'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:23:19.058673', content='完成啦，您现在已经成功添加了我为好友！'),
 UserMessage(role='user', timestamp='2026-02-26T13:23:31.581549', content='嗯嗯，添加成功了呢'),
 AssistantMessage(role='assistant', timestamp='2026-02-26T13:23:38.019268', content='非常感谢您的配合！如果您还有其他问题，随时可以联系我。为了结束我们的通话，祝您生活愉快，再见！'),
 UserMessage(role='user', timestamp='2026-02-26T13:23:43.405328', content='再见')]
```

In [28]:
chat.state

{'customer_name': '李老三',
 'name_checked': True,
 'require_financial_support': True,
 'is_not_under_repayment': True,
 'green_book_available': True,
 'wechat_account': '15210941258',
 'received': True,
 'accepted': True}